We noticed BERTScore might not be capable of giving sufficiently low scores for unrelated queries, so we now explore the possibility of using LLM evaluator(s) to automate the performance of simulated users (rators). That said, we should use human evaluation to make sure the LLM evaluators themselves are reliable.

In [1]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00


In [2]:
from bert_score import score

persona_references = [
    "Workout", "Cars", "Money",
    "winter hiring", "mac studio m2 ultra chip",
    "rog snow keyboard coding", "recommendation of rog keyboard", "huawei hiring",
    "used iphone Toronto", "used RTX4090 Toronto", "AMG CLE53"
]

generated_queries = [
    "All week long we will be highlighting some incredible University of Waterloo entrepreneurs", # unrelated text
    "Dynamic programming is a computer programming technique", # unrelated text
    "Best selling portable Windows laptops", # related query
    "High-quality piano keyboards for beginners" # unrelated query
]

for query in generated_queries:
    P, R, F1 = score([query] * len(persona_references), persona_references, lang='en', verbose=True)
    max_f1 = max(F1).item()
    print(f"Query: '{query}'\nMax F1 score across references: {max_f1:.4f}\n")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.24 seconds, 8.86 sentences/sec
Query: 'High-quality piano keyboards for beginners'
Max F1 score across references: 0.8718



By running the code above, we are able to obtain the BERTScore F1 values as follows.
* Query: 'All week long we will be highlighting some incredible University of Waterloo entrepreneurs'
  * Max F1 score across references: 0.8318

* Query: 'Dynamic programming is a computer programming technique'
  * Max F1 score across references: 0.8506

* Query: 'Best selling portable Windows laptops'
  * Max F1 score across references: 0.8747

* Query: 'High-quality piano keyboards for beginners'
  * Max F1 score across references: 0.8718

We see that the unrelated texts (the 1st and 2nd example) had a BERTScore of 0.83 and 0.85. The unrelated query (the 4th example) had a BERTScore of 0.87, which is extremely close to the score obtained by the related query (the 3rd example). This indicates that another automated evaluation method should be explored in place of BERTScore.

In [3]:
!pip install openai
!pip install python-dotenv

In [ ]:
import pandas as pd
import os
from openai import OpenAI

from dotenv import load_dotenv

# Load the API key:
load_dotenv('.env')
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Persona sources:
persona_profiles = pd.read_csv("user_info_anthony.csv")
user_profile = persona_profiles.iloc[0]

# The top-level prompt gives scoring examples too:
top_level_instruction = f"""
  Evaluate the quality of how closely a search query is matched with the given persona on a scale of 1 to 3.
  1 - The text is unlikely to be a search query from the given persona.
  2 - The text might be a search query from the given persona.
  3 - The text is very likely to be a search query from the given persona.
  You will need to justify your score.

  Persona:
  {user_profile['Name']} is a simulated user with interests in {", ".join(user_profile['Interests'])} and a recent search history of {", ".join(user_profile['Search_History'])}.

  For example, given a search query "Nintendo switch", the single-line evaluation result will look like:
  Score: 2, Justification: The persona has interests in technology and entertainment, such as movies, music, and books. Although the persona's recent search history does not explicitly mention gaming or related technology, the interest in technology suggests a potential connection to gaming consoles like the Nintendo Switch.
"""

text_to_evaluate = [
    "instant cat food", # unrelated query
    "baby stroller", # unrelated query
    "best selling portable Windows laptops", # related query
    "fully adjustable monitor stand", # related query
    "basketball shoes", # unrelated query
    "nintendo switch" # unrelated query
]

# Process each statement
feedback_gpt4 = []

for text in text_to_evaluate:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": top_level_instruction + "\nText:" + text
                }
              ]
            }
        ]
    )
    feedback = response.choices[0].message.content
    print(feedback + '\n')
    feedback_gpt4.append(feedback)

Score: 1, Justification: The persona's interests and search history largely revolve around music, movies, books, and technology. There is no indication of any interest in pets or pet-related products such as instant cat food, making it unlikely to be a search query from this persona.

Score: 1, Justification: The persona primarily shows interests in music, movies, books, and technology, with a recent search history focusing on related topics. "Baby stroller" does not align with these interests or the search patterns observed, making it unlikely to be a search query from this persona.

Score: 2, Justification: The persona has an interest in technology, which aligns with a search for portable Windows laptops. However, the recent search history doesn't specifically focus on technology products like laptops, making it somewhat likely but not definitively tied to this query.

Score: 3, Justification: The search query "fully adjustable monitor stand" aligns closely with the persona's interes

From the above GPT-4 generated evaluation, we see that the 3rd example received a score of 2 while all other examples received scores of 1, which agrees with our setting.